In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset,Dataset,random_split


In [ ]:
# Step 1: Load and Preprocess Data
def load_and_preprocess_data():
    # Load the dataset
    df = pd.read_csv('breast-cancer-dataset.csv')
    
    # Handle missing values (replace '#' with appropriate strategy)
    df = df.replace('#', np.nan)
    df = df.dropna()
    
    # Convert categorical variables
    categorical_columns = ['Menopause', 'Breast', 'Metastasis', 'History']
    for col in categorical_columns:
        df[col] = pd.Categorical(df[col]).codes
    
    # One-hot encode Breast Quadrant
    df = pd.get_dummies(df, columns=['Breast Quadrant'])
    
    # Prepare features and target
    features = df.drop(['S/N', 'Year', 'Diagnosis Result'], axis=1)
    target = (df['Diagnosis Result'] == 'Malignant').astype(int)
    
    return features, target